In [1]:
# 导入模块
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator


In [2]:
### 步骤1：创建SparkSession对象
spark = SparkSession.builder.appName('lin_reg').getOrCreate()


In [5]:
### 读取数据集
df = spark.read.csv('G:\LKM\PySark机器学习、自然语言处理与推荐系统\推荐系统\movie_ratings_df.csv', inferSchema=True, header=True)



In [ ]:
# 数据结构
(df.count(), len(df.columns))



In [6]:
#查看字段类型
df.printSchema()



root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)

